In [10]:
from dis import dis
from tkinter.tix import AUTO
from requests import head
import torch
import config
import os
from glob import glob
from shutil import copyfile
import pandas as pd
import re
import numpy as np
from math import sqrt
import math
import tensorflow as tf
from keras import regularizers
from tensorflow.keras import Model, Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from CustomDataset import custom_collate_fn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,TimeDistributed
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers

In [11]:
def AutoEncoder(x):
    # Building the Input Layer
    input_layer = Input(shape =(x.shape[1], ))
    
    # Building the Encoder network
    encoded = Dense(100, activation ='tanh',
                    activity_regularizer = regularizers.l1(10e-5))(input_layer)
    encoded = Dense(50, activation ='tanh',
                    activity_regularizer = regularizers.l1(10e-5))(encoded)
    encoded = Dense(25, activation ='tanh',
                    activity_regularizer = regularizers.l1(10e-5))(encoded)
    encoded = Dense(12, activation ='tanh',
                    activity_regularizer = regularizers.l1(10e-5))(encoded)
    # encoded = Dense(6, activation ='relu')(encoded)
    
    # Building the Decoder network
    decoded = Dense(12, activation ='tanh')(encoded)
    decoded = Dense(25, activation ='tanh')(decoded)
    decoded = Dense(50, activation ='tanh')(decoded)
    decoded = Dense(100, activation ='tanh')(decoded)
    
    # Building the Output Layer
    output_layer = Dense(x.shape[1], activation ='relu')(decoded)

    # Defining the parameters of the Auto-encoder network
    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(optimizer ="adam", loss ="binary_crossentropy")
    
    # Training the Auto-encoder network
    autoencoder.fit(x.values, x.values, batch_size = 256, epochs = 10, shuffle = True)

    # Take only the Encoded Part of the AutoEncoder
    hidden_representation = Sequential()
    hidden_representation.add(autoencoder.layers[0])
    hidden_representation.add(autoencoder.layers[1])
    hidden_representation.add(autoencoder.layers[2])
    hidden_representation.add(autoencoder.layers[3])
    hidden_representation.add(autoencoder.layers[4])

    # Predict the new features using the learned representation
    reduced_features = hidden_representation.predict(x)
    return pd.DataFrame(reduced_features)

In [16]:
data=pd.read_csv(r"D:\Research_Project\My_project_22\input\final_preprocessed_merged\merged.csv")
data.head()
target=data['166']
data=data.drop(['166'],axis=1)
data.head()
print(data.shape)
new_data=AutoEncoder(data)
# data['12']=target
print(data.shape)
print(new_data.shape)
print(target.shape)

(12260, 166)
Epoch 1/10
48/48 [==============================] - 3s 8ms/step - loss: 12.6443
Epoch 2/10
48/48 [==============================] - 0s 8ms/step - loss: 8.4775
Epoch 3/10
48/48 [==============================] - 2s 44ms/step - loss: 6.5336
Epoch 4/10
48/48 [==============================] - 0s 8ms/step - loss: 4.8271
Epoch 5/10
48/48 [==============================] - 0s 8ms/step - loss: 4.0431
Epoch 6/10
48/48 [==============================] - 2s 43ms/step - loss: 3.8035
Epoch 7/10
48/48 [==============================] - 0s 7ms/step - loss: 3.4055
Epoch 8/10
48/48 [==============================] - 0s 7ms/step - loss: 3.4583
Epoch 9/10
48/48 [==============================] - 2s 45ms/step - loss: 3.3827
Epoch 10/10
384/384 [==============================] - 2s 6ms/step
(12260, 166)
(12260, 12)
(12260,)


In [17]:
new_data['12']=target
new_data.to_csv(r"D:\Research_Project\My_project_22\input\final_preprocessed_merged\merged_autoencoder.csv",index=False)

In [18]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :12], sequences[end_ix-1, 12:]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [19]:
# raw_angle_files_1 = glob(os.path.join("D:\Research_Project\My_project_22\input\preprocessed", "*", "*.csv"))
# # print(raw_angle_files_1)
# all_filenames = [i for i in raw_angle_files_1]
# df = pd.concat(map(pd.read_csv, all_filenames),ignore_index=True)
data=pd.read_csv(r"D:\Research_Project\My_project_22\input\final_preprocessed_merged\merged_autoencoder.csv")

data=pd.get_dummies(data,columns=['12'])
data=data.to_numpy()
x,y=split_sequences(data,3)
print(y.shape)

# target=data['26']
# data=data.drop(['26'],axis=1)
x_train,x_test,y_train,y_test= train_test_split(x, y, test_size = 0.2)



# Model building
n_steps=3

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 12), return_sequences=True))
model.add(LSTM(30, activation='relu'))
model.add(Dense(10,activation='softmax'))
# model.add(Embedding(config.EXTRACTED_FEATURES,128))
# model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
# model.add(GlobalMaxPool1D())
# model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
#     # model = Sequential()
#     # model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
#     # model.add(LSTM(10))
#     # model.add(Dense(1, activation='sigmoid'))
#     # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# #     model = tf.keras.Sequential([
# #     tf.keras.layers.Embedding(config.EXTRACTED_FEATURES, 64),
# #     tf.keras.layers.LSTM(64),
# #     tf.keras.layers.Dense(1, activation="sigmoid")
# # ])
#     # x_train = pad_sequences(x_train, 300)
#     # x_test = pad_sequences(x_test, 300)
model.summary()
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(x.shape)
history = model.fit(x_train, y_train,batch_size=config.BATCH_SIZE,epochs=100,validation_data=(x_test,y_test),verbose=1)
results = model.evaluate(x_test, y_test)

(12258, 10)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 50)             12600     
                                                                 
 lstm_1 (LSTM)               (None, 30)                9720      
                                                                 
 dense_36 (Dense)            (None, 10)                310       
                                                                 
Total params: 22,630
Trainable params: 22,630
Non-trainable params: 0
_________________________________________________________________
(12258, 3, 12)
Epoch 1/100
154/154 [==============================] - 25s 143ms/step - loss: 2.2971 - accuracy: 0.1155 - val_loss: 2.2911 - val_accuracy: 0.1183
Epoch 2/100
154/154 [==============================] - 23s 152ms/step - loss: 2.2834 - accuracy: 0.1140 - val_loss: 2.2869 - val_accuracy: 0.

In [ ]:
encoding_dim = 15 
input_img = Input(shape=(784,))
# encoded representation of input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# decoded representation of code 
decoded = Dense(784, activation='sigmoid')(encoded)
# Model which take input image and shows decoded images
autoencoder = Model(input_img, decoded)
# This model shows encoded images
encoder = Model(input_img, encoded)
# Creating a decoder model
encoded_input = Input(shape=(encoding_dim,))
# last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))


In [92]:
def AutoEncoder(x):
    # Building the Input Layer
    input_layer = Input(shape =(x.shape[1], ))
    
    # Building the Encoder network
    # encoded = Dense(100, activation ='sigmoid',
    #                 )(input_layer)
    # encoded = Dense(50, activation ='sigmoid',
    #                 )(encoded)
    # encoded = Dense(25, activation ='sigmoid',
    #                )(encoded)
    encoded = Dense(12, activation ='sigmoid',
                   )(input_layer)
    # encoded = Dense(6, activation ='relu')(encoded)
    
    # Building the Decoder network
    # decoded = Dense(12, activation ='tanh')(encoded)
    decoded = Dense(x.shape[1], activation ='sigmoid')(encoded)
    # decoded = Dense(50, activation ='sigmoid')(decoded)
    # decoded = Dense(100, activation ='sigmoid')(decoded)
    
    # Building the Output Layer
    # output_layer = Dense(x.shape[1], activation ='relu')(decoded)

    # Defining the parameters of the Auto-encoder network
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer ="adam", loss ="binary_crossentropy")
    
    # Training the Auto-encoder network
    autoencoder.fit(x.values, x.values, batch_size = 784, epochs = 15, shuffle = True)

    # Take only the Encoded Part of the AutoEncoder
    hidden_representation = Sequential()
    hidden_representation.add(autoencoder.layers[0])
    # hidden_representation.add(autoencoder.layers[1])
    # hidden_representation.add(autoencoder.layers[2])
    # hidden_representation.add(autoencoder.layers[3])
    # hidden_representation.add(autoencoder.layers[4])

    # Predict the new features using the learned representation
    reduced_features = hidden_representation.predict(0)
    return pd.DataFrame(reduced_features)

In [107]:
def AutoEncoder(x):
    # Building the Input Layer
    input_layer = Input(shape =(x.shape[1], ))
    
    # Building the Encoder network
    encoded = Dense(128, activation ='sigmoid',
                    )(input_layer)
    encoded = Dense(64, activation ='sigmoid',
                    )(encoded)
    encoded = Dense(32, activation ='sigmoid',
                   )(encoded)
    encoded = Dense(12, activation ='sigmoid',
                   )(input_layer)
    encoded = Dense(6, activation ='relu')(encoded)
    
    # Building the Decoder network
    # decoded = Dense(12, activation ='tanh')(encoded)
    
    decoded = Dense(12, activation ='sigmoid')(encoded)
    decoded = Dense(32, activation ='sigmoid')(decoded)
    decoded = Dense(64, activation ='sigmoid')(decoded)
    decoded = Dense(128, activation ='sigmoid')(decoded)
    decoded = Dense(x.shape[1], activation ='sigmoid')(decoded)
    
    # Building the Output Layer
    # output_layer = Dense(x.shape[1], activation ='relu')(decoded)

    # Defining the parameters of the Auto-encoder network
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer ="adam", loss ="mse")
    
    # Training the Auto-encoder network
    autoencoder.fit(x.values, x.values, batch_size = 784, epochs = 15, shuffle = True)

    # Take only the Encoded Part of the AutoEncoder
    hidden_representation = Sequential()
    hidden_representation.add(autoencoder.layers[0])
    hidden_representation.add(autoencoder.layers[1])
    hidden_representation.add(autoencoder.layers[2])
    hidden_representation.add(autoencoder.layers[3])
    hidden_representation.add(autoencoder.layers[4])

    # Predict the new features using the learned representation
    reduced_features = hidden_representation.predict(x)
    return pd.DataFrame(reduced_features)

In [109]:
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model

In [112]:
def AutoEncoder(X):
    n_inputs = X.shape[1]
# define encoder
    input_data_shape= Input(shape=(n_inputs,))
    # encoder level 1
    encoder= Dense(n_inputs*2)(input_data_shape)
    encoder = BatchNormalization()(encoder)
    encoder= LeakyReLU()(encoder)
    # encoder level 2
    encoder= Dense(n_inputs)(encoder)
    encoder= BatchNormalization()(encoder)
    encoder= LeakyReLU()(encoder)
    # bottleneck
    n_bottleneck = round(float(n_inputs) / 2.0)
    bottleneck = Dense(n_bottleneck)(encoder)
    # define decoder, level 1
    decoder = Dense(n_inputs)(bottleneck)
    decoder = BatchNormalization()(decoder)
    decoder = LeakyReLU()(decoder)
    # decoder level 2
    decoder = Dense(n_inputs*2)(decoder)
    decoder = BatchNormalization()(decoder)
    decoder = LeakyReLU()(decoder)
    output = Dense(n_inputs, activation='linear')(decoder)
# define autoencoder model
    model = Model(inputs=input_data_shape, outputs=output)
    # compile autoencoder model
    model.compile(optimizer='adam', loss='mse')
    
    # Training the Auto-encoder network
    model.fit(X.values, X.values, batch_size = 784, epochs = 15, shuffle = True)

    # Take only the Encoded Part of the AutoEncoder
    # hidden_representation = Sequential()
    # hidden_representation.add(autoencoder.layers[0])
    # hidden_representation.add(autoencoder.layers[1])
    # hidden_representation.add(autoencoder.layers[2])
    # hidden_representation.add(autoencoder.layers[3])
    # hidden_representation.add(autoencoder.layers[4])

    # Predict the new features using the learned representation
    # reduced_features = hidden_representation.predict(X)
    # return pd.DataFrame(reduced_features)
    return

In [113]:
raw_angle_files_1 = glob(os.path.join("D:\Research_Project\My_project_22\input\preprocessed", "*", "*.csv"))
# print(raw_angle_files_1)
all_filenames = [i for i in raw_angle_files_1]
df = pd.concat(map(pd.read_csv, all_filenames),ignore_index=True)
#df=pd.read_csv(r"D:\Research_Project\My_project_22\FEATURES_EXTRACTED\DISTANCES\both_hand_frontup_left_leg_frontup\aatish_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv")
   
data=df
print(data.shape)
target=data['54']
data=data.drop(['54'],axis=1)
# data.head()
print(data.shape)
AutoEncoder(data)
data['12']=target
print(data.shape)

(12260, 55)
(12260, 54)
Epoch 1/15
16/16 [==============================] - 1s 17ms/step - loss: 4813699.5000
Epoch 2/15
16/16 [==============================] - 0s 10ms/step - loss: 4813436.0000
Epoch 3/15
16/16 [==============================] - 0s 10ms/step - loss: 4813316.0000
Epoch 4/15
16/16 [==============================] - 0s 10ms/step - loss: 4813317.5000
Epoch 5/15
16/16 [==============================] - 0s 9ms/step - loss: 4813132.5000
Epoch 6/15
16/16 [==============================] - 0s 10ms/step - loss: 4813013.5000
Epoch 7/15
16/16 [==============================] - 0s 10ms/step - loss: 4812929.5000
Epoch 8/15
16/16 [==============================] - 0s 10ms/step - loss: 4812769.5000
Epoch 9/15
16/16 [==============================] - 0s 10ms/step - loss: 4812729.0000
Epoch 10/15
16/16 [==============================] - 0s 10ms/step - loss: 4812640.0000
Epoch 11/15
16/16 [==============================] - 0s 9ms/step - loss: 4812587.0000
Epoch 12/15
16/16 [===========

In [106]:
data.to_csv(r"D:\Research_Project\My_project_22\input\final_preprocessed_merged\merged.csv",index=False)